# Scenario

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including inforamtion like session time, location, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:
    
    1. 'Session_Connection_Time': How long the session lasted in minutes
    2. 'Bytes Transferred': Number of MB transferred during session
    3. 'Kali_Trac_Used': Indicates if hacker was using Kali Linux
    4. 'Servers_Corrupted': Number of server corrupted during the attack
    5. 'Pages_Corrupted': Number of pages ilegally accessed
    6. 'Location': Location attack came from (probably useless because the hackers used VPNs)
    7. 'WPM_Typing_Speed': Their estimated typeing speed based on session logs.
    
The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It is probably not possible to know for sure, but maybe what you have learned about Clustering can help!

One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('hack_find').getOrCreate()

In [3]:
from pyspark.ml.clustering import KMeans

In [4]:
# Load data
dataset = spark.read.csv('hack_data.csv',header=True, inferSchema = True)

In [5]:
dataset.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [6]:
dataset.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [8]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [10]:
feat_cols = ['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used',
             'Servers_Corrupted', 'Pages_Corrupted','WPM_Typing_Speed']

In [11]:
vec_assembler = VectorAssembler(inputCols = feat_cols, outputCol='features')

In [12]:
final_data = vec_assembler.transform(dataset)

In [13]:
from pyspark.ml.feature import StandardScaler

In [14]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [15]:
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(final_data)

In [16]:
# Normalize each feature to have unit standard deviation.
cluster_final_data = scalerModel.transform(final_data)

In [17]:
kmeans3 = KMeans(featuresCol='scaledFeatures',k=3)
kmeans2 = KMeans(featuresCol='scaledFeatures',k=2)

In [18]:
model_k3 = kmeans3.fit(cluster_final_data)
model_k2 = kmeans2.fit(cluster_final_data)

In [20]:

# Make predictions
predictions3 = model_k3.transform(cluster_final_data)
predictions2 = model_k2.transform(cluster_final_data)

In [22]:
from pyspark.ml.evaluation import ClusteringEvaluator
# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

In [26]:
silhouette = evaluator.evaluate(predictions3)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model_k3.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Silhouette with squared euclidean distance = 0.3068084951287429
Cluster Centers: 
[2.99991988 2.92319035 1.05261534 3.20390443 4.51321315 3.28474   ]
[1.21780112 1.37901802 1.99757683 1.37198977 2.55237797 5.29152222]
[1.30217042 1.25830099 0.         1.35793211 2.57251009 5.24230473]


In [27]:
silhouette2 = evaluator.evaluate(predictions2)
print("Silhouette with squared euclidean distance = " + str(silhouette2))

# Shows the result.
centers = model_k2.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Silhouette with squared euclidean distance = 0.6683623593283755
Cluster Centers: 
[1.26023837 1.31829808 0.99280765 1.36491885 2.5625043  5.26676612]
[2.99991988 2.92319035 1.05261534 3.20390443 4.51321315 3.28474   ]


In [28]:
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   83|
|         2|   84|
|         0|  167|
+----------+-----+



In [29]:
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



Bingo! It was 2 hackers, in fact, our clustering algorithm created two equally sized clusters with K=2, no way that is a coincidence